# Force

## Index

- [Definition](#Definition)
- [List of defined Forces](#List-of-defined-Surfaces)
- [Example](#Example)
- [Adding Forces](#Force-summation)
- [Evaluation](#Evaluation)
- [Creating new Force](#Creating-new-surface)
    - [Example : MyForce]
- [Conservative Force](#Conservative-Force)
- [Interactions from forces](#Interactions)

## Definition

In the Newton approach a *force* is a quantity that tends to accelerate an object, the variation of the momentum of a solid of mass $m$ equals the sum of all forces applying on the solid ([Newton second law of motion](https://en.wikipedia.org/wiki/Newton%27s_laws_of_motion)) :

$$ \frac{d}{dt} \left(m \mathbf{v} \right) = \mathbf{F}$$

All forces can represented by the class `gsurface.force.force.Force`, moreover forces that are [convervative](https://en.wikipedia.org/wiki/Conservative_force) need to be defined using the `gsurface.force.force.ConservativeForce` class which implements the [potential](https://en.wikipedia.org/wiki/Scalar_potential) calculation.

Forces that are not conservatives are [dissipative forces](https://en.wikipedia.org/wiki/Dissipation) represented by the base `.Force` class.

## List of defined Forces

Following basic forces are already completly defined in the library (`gsurface.force` sub-package) :



**Dissipative Forces** : [**Friction Forces**](https://en.wikipedia.org/wiki/Friction) : `.friction`
- Solid friction `.SolidFriction` with parameter $intensity (N)$
- [Viscous friction](https://en.wikipedia.org/wiki/Viscosity) 
    - Motion Directed Viscous Friction`.ViscousFriction` with parameter viscosity $\mu (N.s.m^{-1})$
    - Reference Directed Viscous Friction `.DirectedViscousFriction`
        - Constant-Direction Directed Viscous Friction `.ConstantDirectedViscousFriction` with direction parameter $\vec{D}$
        - Center-Direction Directed Viscous Friction `.CenterDirectedViscousFriction` with center point parameter $\vec{O}$
- [High velocity Aerodynamic Friction - Drag Equation](https://en.wikipedia.org/wiki/Drag_(physics)) `.AirFriction` with parameters : 
    - volumic mass $\rho (kg.m^{-3})$
    - reference area $S (m^2)$
    - [lift coefficient](https://en.wikipedia.org/wiki/Lift_coefficient) $C_x$.


**Conservative Forces**
- Gravity Forces `.gravity`
    - [Constant Gravity](https://en.wikipedia.org/wiki/Gravity_of_Earth) `Gravity` with parameter mass $m (kg)$ and gravity acceleration $g (m.s^{-2})$
    -[NewtonGravity](https://en.wikipedia.org/wiki/Newton%27s_law_of_universal_gravitation) `NewtonGravity` with parameters :
        - masses $m (kg), M (kg)$
        - [Gravitational Constant](https://en.wikipedia.org/wiki/Gravitational_constant) $\mathcal{G}$
        - center of the mass $M$ : $\vec{O}$
- Electromagnetic Forces `.em`
    - [ElctromaMagneticField](https://en.wikipedia.org/wiki/Electromagnetic_field) Force `.StaticFieldElectroMagneticForce` with parameters [charge](https://fr.wikipedia.org/wiki/Charge_%C3%A9lectrique) $q (C)$, [electric field](https://en.wikipedia.org/wiki/Electric_field) $\mathbf{E} (V.m^{-1})$ and magnetic field $\mathbf{B} (T)$
    - **todo** `.MagnetForce`
- Spring Force `spring`
    - [Spring](https://en.wikipedia.org/wiki/Hooke%27s_law) Force `.SpringForce` with parameter stiffness $k (N.m^{-1})$.
    - Spring Lengthed Force  `.LengthedSpringForce` with parameters length at rest $l_0 (m)$ and stiffness $k (N.m^{-1})$.

**Special types Forces (combination)** `.special`
- Spring Damping Force `.SpringDampingForce`


# Example

We defined the following force as a sort of *force field* :

$$
\mathbf{F} (t, \mathbf{S}, \mathbf{V}) = \begin{bmatrix}
1 \\
2 \\
3 \\
\end{bmatrix}
$$

# Force summation

In order to take into account all forces that applies to the solid. Forces can be sum up as described bellow.

$$F_{tot} = F_{spring} + F_{gravity} + F_{friction}$$

For example the following script how to represent the sum of tree forces that applies to a solid :
- Spring force (clipped at origin)
- Gravity Force 
Programatical way :

In [1]:
from gsurface import Solid
from gsurface.forces import SpringForce, Gravity, ViscousFriction

s = Solid(1.0)

F = SpringForce(10.0) + Gravity(s, [0.0, 0.0, -9.81]) + ViscousFriction(0.5)

# Evaluation

The force can be evaluated at any time $t$ when position $\mathbf{S}$ and speed $\mathbf{V}$ are well devined : $\mathbf{F}(t, \mathbf{S}, \mathbf{V})$

In [2]:
import numpy as np

F.eval(t=0.0,  S=np.array([1.0, 0.0, 0.0]), V=np.array([1.0, 1.0, 0.0]))

array([-10.5 ,  -0.5 ,  -9.81])

# Creating new Force

`Force` and `ConservativeForce` can be extended in order to build new/missing/special forces.


In [9]:
from gsurface.forces import Force
    
class MyForce(Force):
    a = 1.0 # N m^-2
    
    def __init__(self, a = 1.0):
        self.a = float(1.0)
    
    def eval(self, t: float, S: np.ndarray = None, V: np.ndarray = None) -> np.ndarray:
        return -self.param*t*S*V
    
    __repr_str__ = "a = {a:.2f} N m^-2"
    
mf = MyForce(2.0)

print(mf)

MyForce : a = 1.00 N m^-2


# Conservative Force

[Conservative forces](https://en.wikipedia.org/wiki/Conservative_force) are represented by the class `ConservativeForce`.

In [5]:
from gsurface.forces import ConservativeForce

class MyConservativeForce(ConservativeForce):
    def __init__(param: float = 1.0, **kargs):
        self.param = float(param)
        
    def eval(self, t: float, S: np.ndarray = None, V: np.ndarray = None) -> np.ndarray:
        return - t * self.param * S
    
    def potential(self, t: float, S: np.ndarray) -> float:
        pass

# Interactions